In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
csv = "traveltime_results_top1_all.csv"
df = pd.read_csv(csv)
df

In [ ]:
csv_shapes = "desd-4-landkreis-hildesheim-1663168323029-shapes.csv"
df_shapes = pd.read_csv(csv_shapes)
df_shapes

In [ ]:
df_shapes.id.describe()

In [ ]:
df_merged = df_shapes.merge(df, how='outer', left_on='name', right_on='search_id')
df_merged

In [ ]:
df_merged.fillna(0, inplace=True)
df_merged

In [ ]:
df_merged['search_id_y'].unique()

In [ ]:
df_merged.to_csv(csv_shapes.split('.')[0] + '.csv', index=False)

In [ ]:
dbf = "desd-4-landkreis-hildesheim-1663168323029-shapes.dbf"
gdf = gpd.read_file(dbf)
gdf

In [ ]:
gdf['search_id'].unique()

In [ ]:
df_merged = gdf.merge(df_shapes, how='left', left_index=True, right_index=True, suffixes=['_dbf', '_csv'])
df_merged

In [ ]:
df_merged['search_id_csv'].unique()

In [ ]:
df_merged.fillna(0, inplace=True)
df_merged

In [ ]:
gdf = gpd.GeoDataFrame(df_merged, geometry='geometry')
gdf.crs = 'EPSG:4326'
gdf

In [ ]:
unique_values = gdf['id_csv'].unique()

integer_labels = pd.factorize(unique_values)[0]

# Create a mapping dictionary for unique values and their integer labels
mapping = dict(zip(unique_values, integer_labels))

# Map the integer labels to the 'id' column in the DataFrame
gdf['id_integer'] = gdf['id_csv'].map(mapping)

gdf

In [ ]:
gdf['id_integer'].describe()

manual validity checking

In [ ]:
cluster = {'Hagenburg': 'Sarstedt (31157 - 03254028001003)',
           'Springe (31832 - 03241017060003)': 'Pattensen (30982 - 03241013050007)',
           'Springe (31832 - 03241017060002)': 'Pattensen (30982 - 03241013050007)',
           #'Springe (31832 - 03241017010001)': 'Pattensen (30982 - 03241013050007)',
           'Gronau (31028 - 03254043000001)': 'Elze (31008 - 03254014000004)',
           'Bockenem (31167 - 03254008000007)': 'Hildesheim (31137 - 03254021003014)',

        }

In [ ]:
# Custom function to update id based on cluster dictionary
def update_id(row):
    return cluster.get(row['search_id_csv'], row['id_csv'])

# Apply the custom function to the DataFrame
gdf['id_csv'] = gdf.apply(update_id, axis=1)
gdf

In [ ]:
gdf['id_integer'] = gdf['id_csv'].map(mapping)
gdf

In [ ]:
gdf['search_id_csv'].unique()

In [ ]:
non_zero_count = (gdf['search_id_csv'] != '0').sum()
non_zero_count

In [ ]:
# Assuming your DataFrame is named 'gdf'
duplicate_rows = gdf[gdf.duplicated(subset=['search_id_csv'], keep=False)]
duplicate_rows

In [ ]:
gdf.loc[1331, ['id_integer', 'id_csv', 'search_id_csv']] = 0

In [ ]:
gdf.loc[1332, ['id_integer', 'id_csv', 'search_id_csv']] = 0

In [ ]:
gdf.to_file('desd-4-landkreis-hildesheim-1663168323029-shapes.dbf', index=False)

In [ ]:
csv = "combined_data_MP_NE_dT_cC.csv"
df = pd.read_csv(csv)
df